In [1]:
# 基础的asm文件，用于构建CFG
fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/static analysis/hw-memory/inputs/small/small.asm"

# 带bss段和data段的asm文件，用于查看全局变量地址空间
seg_fp = r'/Users/gugujixiao/workspace/project/HWMemory/Code/static analysis/hw-memory/inputs/small/smallD.asm'

# 带源码的asm文件，用于源码和汇编代码对照
code_fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/static analysis/hw-memory/inputs/small/smallL.asm"


In [2]:
'''读指令, 指令对象化'''
from cfg.cfg import proc_identify
from cfg.isa import Instruction, AddrMode
from typing import Tuple
from cfg.read_asm import AsmFileReader, StatementType
from cache_analysis.read_segment import segmentReader
from cfg.read_codeMix import AsmCodeReader, MixStatementType

reader =AsmFileReader(fp)
segreader = segmentReader(seg_fp)
codereader = AsmCodeReader(code_fp)

statements = list()
code_stat = list()

for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

for i in range(len(codereader.statements)):
    s: Tuple[MixStatementType, tuple]
    if codereader.statements[i][0] == MixStatementType.codeline:
        codeline = codereader.statements[i][1]
        addr = ""
        not_find = True
        k = i
        while not_find:
            k += 1
            if codereader.statements[k][0] == MixStatementType.asm:
                addr =  int(codereader.statements[k][1][0],16) 
                not_find = False

        code_stat.append((addr,codeline,"loopname"))

procs = proc_identify(statements)
# for proc in procs:
#     print(proc.name, proc.beg_addr.hex_str())
#     if False and proc.name == 'main':
#         for inst in proc.instruction:
#             print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
# print()

In [3]:
'''分proc'''
from graphviz import Digraph
from cfg.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output/procedures.svg'

In [4]:
'''处理call_graph'''
from cfg.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output/call_graph.svg'

In [5]:
'''生成TCFG和loop信息'''
from cfg.cfg import TCfg


tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()
tcfg.build_loop_hrchy()
# tcfg.add_loop_bound(r'D:\workspace\hw-memory\benchmarks\loop_bound.txt')
# tcfg.add_loop_bound(r'D:\workspace\Gitdocuments\hw-memory\benchmarks\loop_bound.txt')

loop_range =list()
with open("./output/cfg_addr.txt","w") as f:
    for l in tcfg.loops:
        #f.write(l.name+"\n")
        #print(l.name)
        for n in l.all_nodes:
            ins = n.instructions
            #head = ins[0].addr
            head,end = n.inst_range
            string = l.name[1:]+" "+head.hex_str()+"-"+end.hex_str()+"\n"
            loop_range.append(([head.val(),end.val()],l.name))
            f.write(string)
        f.write("\n")


loop_code_range =list()


for i in code_stat:
    for l in loop_range:
        if l[0][0]<i[0]<l[0][1]:
            loop_code_range.append((i[0],i[1],l[1]))

#for i in loop_code_range:
#    print(i)


for l in tcfg.loops:
    print("{} (in {} children {}): {}->{} {}".format(l.name, l.father.name if l.father is not None else "None", [sub_l.name for sub_l in l.children], l.back_edge.src.name, l.back_edge.dst.name, [n.name for n in l.all_nodes]))
    l.back_edge.is_backEdge = True
for n in tcfg.all_nodes:
     temp = n.inside_loop.name if n.inside_loop is not None else 'None'
     print("{} {}".format(n.name, temp))



#for n in tcfg_nodes:
#    print(n.name,"n的值",n.node_value)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)




l2 (in l1 children []): n104->n105 ['n105', 'n104']
l1 (in l0 children []): n106->n107 ['n107', 'n106', 'n105', 'n104', 'n103']
l6 (in l3 children []): n93->n94 ['n94', 'n93']
l12 (in l11 children []): n83->n84 ['n84', 'n83']
l11 (in l7 children []): n85->n86 ['n86', 'n85', 'n84', 'n83', 'n82']
l19 (in l13 children []): n74->n75 ['n74', 'n75']
l28 (in l27 children []): n65->n66 ['n65', 'n66']
l27 (in l20 children []): n67->n68 ['n65', 'n64', 'n68', 'n67', 'n66']
l38 (in l29 children []): n56->n57 ['n56', 'n57']
l49 (in l39 children []): n46->n47 ['n43', 'n44', 'n45', 'n46', 'n47']
l61 (in l50 children []): n35->n36 ['n33', 'n34', 'n35', 'n36', 'n113', 'n32']
l74 (in l62 children []): n25->n26 ['n23', 'n24', 'n25', 'n26', 'n112', 'n22']
l0 (in None children []): n108->n109 ['n108', 'n107', 'n106', 'n105', 'n104', 'n103', 'n102', 'n109']
l3 (in None children []): n95->n96 ['n96', 'n95', 'n94', 'n93', 'n92']
l7 (in None children []): n87->n88 ['n86', 'n85', 'n84', 'n83', 'n82', 'n81', 'n8

In [6]:
from rw_analysis.rwcond_out import loadstore_Obj
lds_obj = loadstore_Obj(segreader,tcfg)
lsproc = lds_obj.lsproc
ls_loop_info = lds_obj.loop_info

lds_table = lds_obj.lsproc.ls_table

for i in lds_table:
    if i.target_num == 1:
        print(i.ins.tokens,i.reg_target,i.final_addr,i.addr_offset,"是否找到",i.is_find,i.node.name) 
    else:
        print(i.ins.tokens,i.reg_target_list[0],i.reg_target_list[1],i.addr_offset,i.final_addr,"是否找到",i.is_find,i.node.name)


#for k,v in ls_loop_info.items():
#    print(k,v)

('4006cc', 'b9001fe0', 'str', None, 'w0, [sp, #28]') sp 0 28 是否找到 True n0
('4006d0', 'f9000be1', 'str', None, 'x1, [sp, #16]') sp 0 16 是否找到 True n0
('4006dc', 'b900001f', 'str', None, 'wzr, [x0]') 6fc000 7325440 1792 是否找到 True n0
('4006e4', 'b9002fe0', 'str', None, 'w0, [sp, #44]') sp 0 44 是否找到 True n0
('4006e8', 'b9002bff', 'str', None, 'wzr, [sp, #40]') sp 0 40 是否找到 True n0
('4006ec', 'b9402fe0', 'ldr', None, 'w0, [sp, #44]') sp 0 44 是否找到 True n0
('400718', 'b9000001', 'str', None, 'w1, [x0]') 6fc000 7325440 1792 是否找到 True n5
('40071c', 'b9402be0', 'ldr', None, 'w0, [sp, #40]') sp 0 40 是否找到 True n5
('400738', 'b9000001', 'str', None, 'w1, [x0]') 6fc000 7325440 1792 是否找到 True n7
('400754', 'b9000001', 'str', None, 'w1, [x0]') 6fc000 7325444 1796 是否找到 True n9
('40076c', 'b9000001', 'str', None, 'w1, [x0]') 6fc000 7325440 1792 是否找到 True n11
('400788', 'b9000001', 'str', None, 'w1, [x0]') 6fc000 7325444 1796 是否找到 True n13
('4007ac', 'b9000001', 'str', None, 'w1, [x0]') 6fc000 7325444 179

In [7]:
# from newCFG.heat_analysis import loop_heat
# test=loop_heat(tcfg,lsproc,r'C:\Users\13377\Desktop\hw-memory-master (5)\hw-memory-master\benchmarks\final_benchmark\spec_benchD.asm')
# test.do_it()


In [8]:

# TODO.
from cache_analysis.cache_risk_level import CacheRisk

cache_test = CacheRisk(tcfg, lsproc, r'D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmarkD.asm',
                       r'D:\workspace\hw-memory\cache_analysis\new_cache\input\cache_information.in')
cache_test.test()


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\workspace\\hw-memory\\benchmarks\\final_benchmark\\spec_benchmarkD.asm'

In [ ]:

# from newCFG.rw_condition import RWProc, RWType
# from newCFG.rw_condition_out import RWOut_Proc



# rwout = RWOut_Proc(tcfg_nodes,segreader,rwproc.rw_table,tcfg.loops)




